In [46]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

### Load the Collisions data

Pleasanton City Police Department provided data for traffic collision incidents in Pleasanton, between 2010 and 2019.

In [47]:
# Convert to a DataFrame and render.
incidents_df = pd.DataFrame.from_csv("./data/CollisionData01012010-02012019_reduced_columns - Sheet1.csv")

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


#### Show the columns

In [48]:
incidents_df.columns

Index(['Date', 'Time', 'Collision Day of Week', 'Primary Road',
       'Secondary Road', 'X', 'Y', 'Latitude', 'Longitude',
       'Highest Degree of Injury', 'Number Injured', 'Number Killed',
       'Party at Fault', 'Primary Collision Factor', 'Hit and Run',
       'Collision Type', 'Involved With', 'Pedestrian Action', 'Violation',
       'Violation Subsection', 'Weather', 'Road Condition1', 'Lighting',
       'Right of Way Controls', 'Party Type 1', 'Party Age 1', 'Party Sex 1',
       'Movement Preceding Collision 1', 'special information 1',
       'Associated Cause - Factor 1 1', 'Party Sobriety 1 1', 'Party Type 2',
       'Party Age 2', 'Party Sex 2', 'Movement Preceding Collision 2',
       'special information 2', 'Associated Cause - Factor 1 2',
       'Party Sobriety 1 2', 'Safety Equipment 2'],
      dtype='object')

#### Explore the data

In [49]:
incidents_df['Collision Day of Week'].unique()

array(['Saturday', 'Tuesday', 'Wednesday', 'Sunday', 'Monday', 'Thursday',
       'Friday'], dtype=object)

#### Add a new column "Injuries" derived from "Number Injured"

In [50]:
incidents_df["Injuries"] = incidents_df["Number Injured"][1:].astype(int)
incidents_df[["Injuries"]]

,Injuries
Report Number,
201100000107,NaN
2013-00000039,0.0
201400000078,3.0
201700000025,0.0
201700000090,2.0
201100001479,1.0
201200001132,0.0
201200001142,1.0
2013-00001139,0.0


### Custom Functions

In [51]:
import time as t
from datetime import datetime

In [52]:
def combine_date_time(date,time):
    date_time_str = datetime.strptime(date,"%m/%d/%Y")       
    if str(time) != "" and str(time) != "nan":
        time_val = datetime.strptime(time,"%H:%M").time()
        date_time = datetime.combine(date_time_str,time_val)
    return date_time

In [53]:
def is_schooltime(date,time):
    school_hour_start_time = datetime.strptime("07:30","%H:%M").time()
    school_hour_end_time  = datetime.strptime("15:30","%H:%M").time()
    if str(time) != "" and str(time) != "nan":
        time_val = datetime.strptime(time,"%H:%M").time()
        if time_val is not None and time_val > school_hour_start_time and time_val < school_hour_end_time:
            return 1
        else:
            return 0
    else:
        return 0

### Data Filtered for Santa Rita Rd

In [54]:
road_injury_santa_rita_df = incidents_df[incidents_df["Primary Road"].isin(["SANTA RITA RD"])][["Primary Road","Secondary Road"]].groupby(["Secondary Road"]).count()
road_injury_santa_rita_df.columns = ["Primary_Santa_Rita_Rd"]
road_injury_santa_rita_df.sort_values("Primary_Santa_Rita_Rd",ascending=False)

,Primary_Santa_Rita_Rd
Secondary Road,
VALLEY AV,59
STONERIDGE DR,42
MOHR AV,42
LAS POSITAS BL,36
PIMLICO DR,35
ROSEWOOD DR,27
FRANCISCO ST (W),20
LOCKHART LN,18
BLACK AV (W),15


In [55]:
schooltime_injury_santa_rita_df = incidents_df[incidents_df["Primary Road"].isin(["SANTA RITA RD"])].copy()
schooltime_injury_santa_rita_df["SchoolHour"] = schooltime_injury_santa_rita_df.apply(lambda x: is_schooltime(x['Date'],x['Time']),axis=1)

In [56]:
schooltime_injury_santa_rita_df

,Date,Time,Collision Day of Week,Primary Road,Secondary Road,X,Y,Latitude,Longitude,Highest Degree of Injury,...,Party Type 2,Party Age 2,Party Sex 2,Movement Preceding Collision 2,special information 2,Associated Cause - Factor 1 2,Party Sobriety 1 2,Safety Equipment 2,Injuries,SchoolHour
Report Number,,,,,,,,,,,,,,,,,,,,,
201100001479,1/10/2011,11:47,Monday,SANTA RITA RD,VALLEY AV,6164052.164,2071822.231,37.676806,-121.873915,Property Damage Only,...,Driver,65.0,Not Stated,Stopped in Road,Not Stated,Not Stated,HNBD,Lap/Shoulder Harness Used,1.0,1
201500001561,1/12/2015,18:59,Monday,SANTA RITA RD,MOHR AV,6163423.095,2074241.199,37.683472,-121.876283,Property Damage Only,...,Driver,60.0,Male,Proceeding Straight,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,0.0,0
201400001828,1/13/2014,15:09,Monday,SANTA RITA RD,MOHR AV,6163556.925,2073723.208,37.682027,-121.875626,Other Visible Injury,...,Driver,NaN,Male,Proceeding Straight,Cell Phone Not In Use,None Apparent,HNBD,Lap Belt Used,1.0,1
2013-00002304,1/17/2013,22:22,Thursday,SANTA RITA RD,PIMLICO DR,6164710.000,2080338.000,37.700299,-121.872111,Property Damage Only,...,Driver,25.0,Female,Making Left Turn,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,0.0,0
201100003303,1/19/2011,12:30,Wednesday,SANTA RITA RD,SCHOOL ST,6164272.964,2068688.285,37.668147,-121.873125,Property Damage Only,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1
201200002545,1/19/2012,12:53,Thursday,SANTA RITA RD,LAS POSITAS BL,6162819.370,2077348.002,37.692004,-121.878370,Complaint of Pain,...,Driver,47.0,Female,Stopped in Road,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,1.0,1
201700002210,1/19/2017,17:42,Thursday,SANTA RITA RD,VALLEY AV,6164066.000,2071769.000,37.676660,-121.873867,Complaint of Pain,...,Driver,36.0,Male,Stopped In Road,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,0.0,0
201800000164,1/2/2018,10:00,Tuesday,SANTA RITA RD,OLD SANTA RITA RD,6162934.943,2078307.234,37.694557,-121.878231,Complaint of Pain,...,Driver,22.0,Female,Stopped In Road,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,1.0,1
201100003306,1/20/2011,10:17,Thursday,SANTA RITA RD,VALLEY AV,6164066.000,2071769.000,37.676660,-121.873867,Complaint of Pain,...,Driver,44.0,Female,Proceeding Straight,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,2.0,1


#### Sant Rita Rd Report

##### View Specific Columns

In [57]:
target_df = schooltime_injury_santa_rita_df[["SchoolHour","Number Injured","Collision Type","Party at Fault","Party Type 1","Party Age 1","Party Type 2","Party Age 2","Primary Road","Secondary Road"]].sort_values(["Number Injured"],ascending=False)
target_df

,SchoolHour,Number Injured,Collision Type,Party at Fault,Party Type 1,Party Age 1,Party Type 2,Party Age 2,Primary Road,Secondary Road
Report Number,,,,,,,,,,
201000046623,0,5,Rear-End,NaN,Driver,37.0,Driver,64.0,SANTA RITA RD,LOCKHART LN
201700045379,0,4,Rear-End,1.0,Driver,17.0,Driver,53.0,SANTA RITA RD,MOHR AV
201300022949,1,4,Broadside,1.0,Driver,44.0,Driver,51.0,SANTA RITA RD,LAS POSITAS BL
201700032464,0,4,Rear-End,1.0,Driver,43.0,Driver,17.0,SANTA RITA RD,STONERIDGE DR
201300038422,0,4,Head-On,1.0,Driver,49.0,Driver,54.0,SANTA RITA RD,SCHOOL ST
201000028576,0,3,Rear-End,NaN,Driver,48.0,Driver,41.0,SANTA RITA RD,LOCKHART LN
201600006971,0,3,Head-On,1.0,Driver,43.0,Driver,86.0,SANTA RITA RD,LAS POSITAS BL
201500011861,1,3,Rear-End,1.0,Driver,79.0,Driver,53.0,SANTA RITA RD,MOHR AV
201000016072,0,3,Rear-End,NaN,Driver,34.0,Driver,56.0,SANTA RITA RD,MOHR AV


##### Pedestrian Involved Injuries on Santa Rita Rd

In [58]:
target_df[(target_df["Party Type 1"] == "Pedestrian") | (target_df["Party Type 2"] == "Pedestrian")]

,SchoolHour,Number Injured,Collision Type,Party at Fault,Party Type 1,Party Age 1,Party Type 2,Party Age 2,Primary Road,Secondary Road
Report Number,,,,,,,,,,
201600026126,1,2,Vehicle - Pedestrian,1.0,Driver,NaN,Pedestrian,32.0,SANTA RITA RD,LAS POSITAS BL
201300036051,0,2,Vehicle - Pedestrian,1.0,Driver,49.0,Pedestrian,72.0,SANTA RITA RD,FRANCISCO ST (W)
201300017867,1,1,Vehicle - Pedestrian,NaN,Driver,NaN,Pedestrian,48.0,SANTA RITA RD,SUTTER GATE AV
201400011237,1,1,Vehicle - Pedestrian,1.0,Driver,NaN,Pedestrian,16.0,SANTA RITA RD,VALLEY AV
201000040320,1,1,Vehicle - Pedestrian,NaN,Driver,45.0,Pedestrian,16.0,SANTA RITA RD,VALLEY AV
201200030042,0,1,Vehicle - Pedestrian,1.0,Driver,NaN,Pedestrian,11.0,SANTA RITA RD,VALLEY AV
201400032221,1,1,Vehicle - Pedestrian,1.0,Driver,45.0,Pedestrian,69.0,SANTA RITA RD,FRANCISCO ST (W)
201200029464,0,1,Vehicle - Pedestrian,1.0,Pedestrian,24.0,Driver,29.0,SANTA RITA RD,SUTTER GATE AV
201600002813,1,1,Vehicle - Pedestrian,1.0,Driver,22.0,Pedestrian,12.0,SANTA RITA RD,SILVER ST (W)


#####  Parties involved less than 16 years Age

In [59]:
target_df[(target_df["Party Age 1"] < 16) | (target_df["Party Age 2"] < 16)]

,SchoolHour,Number Injured,Collision Type,Party at Fault,Party Type 1,Party Age 1,Party Type 2,Party Age 2,Primary Road,Secondary Road
Report Number,,,,,,,,,,
201800018647,0,1,Broadside,1.0,Driver,54.0,Bicyclist,12.0,SANTA RITA RD,BLACK AV (W)
201013242,1,1,Overturned,NaN,Bicyclist,12.0,NaN,NaN,SANTA RITA RD,ALVARADO ST
201300031391,0,1,Broadside,1.0,Driver,81.0,Bicyclist,13.0,SANTA RITA RD,OLD SANTA RITA RD
201100015352,0,1,Broadside,1.0,Driver,40.0,Bicyclist,13.0,SANTA RITA RD,FRANCISCO ST (W)
201500026561,1,1,Broadside,1.0,Driver,86.0,Bicyclist,12.0,SANTA RITA RD,FRANCISCO ST (W)
201200030042,0,1,Vehicle - Pedestrian,1.0,Driver,NaN,Pedestrian,11.0,SANTA RITA RD,VALLEY AV
201800035353,1,1,Broadside,1.0,Driver,42.0,Bicyclist,11.0,SANTA RITA RD,LAS POSITAS BL
201400039240,1,1,Broadside,1.0,Bicyclist,11.0,Driver,82.0,SANTA RITA RD,VALLEY AV
201600002813,1,1,Vehicle - Pedestrian,1.0,Driver,22.0,Pedestrian,12.0,SANTA RITA RD,SILVER ST (W)


##### School Time Injuries

In [60]:
road_injury_date_time_df = schooltime_injury_santa_rita_df[["Primary Road", "Secondary Road", "Date", "Time","Number Injured","Number Killed"]]
road_injury_date_time_df["Date"] = pd.to_datetime(road_injury_date_time_df["Date"][1:])
road_injury_date_time_df.sort_values(["Date","Time"],ascending=False)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Primary Road,Secondary Road,Date,Time,Number Injured,Number Killed
Report Number,,,,,,
201900003017,SANTA RITA RD,PIMLICO DR,2019-01-26,17:09,0,0.0
201900002949,SANTA RITA RD,SILVER ST (W),2019-01-26,01:59,1,0.0
201900000591,SANTA RITA RD,LAS POSITAS BL,2019-01-06,17:34,1,0.0
201800046978,SANTA RITA RD,MOHR AV,2018-12-13,17:54,0,0.0
201800045695,SANTA RITA RD,SILVER ST (W),2018-12-04,11:30,0,0.0
201800043691,SANTA RITA RD,LAS POSITAS BL,2018-11-16,22:05,1,0.0
201800043520,SANTA RITA RD,ROSEWOOD DR,2018-11-15,09:11,1,0.0
201800043307,SANTA RITA RD,BLACK AV (W),2018-11-13,15:23,1,0.0
201800042062,SANTA RITA RD,LOCKHART LN,2018-11-02,12:52,0,0.0


### Data Filtered for Valley Ave

In [61]:
road_injury_df = incidents_df[incidents_df["Primary Road"].isin(["VALLEY AV"])]

In [62]:
road_injury_df

,Date,Time,Collision Day of Week,Primary Road,Secondary Road,X,Y,Latitude,Longitude,Highest Degree of Injury,...,Party Sobriety 1 1,Party Type 2,Party Age 2,Party Sex 2,Movement Preceding Collision 2,special information 2,Associated Cause - Factor 1 2,Party Sobriety 1 2,Safety Equipment 2,Injuries
Report Number,,,,,,,,,,,,,,,,,,,,,
201600001058,1/10/2016,18:00,Sunday,VALLEY AV,BLACKBIRD DR,6158417.000,2071864.000,37.676619,-121.893386,Complaint of Pain,...,Sleepy - Fatigued,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
201700000989,1/10/2017,08:50,Tuesday,VALLEY AV,BUSCH RD,6167305.341,2071219.108,37.675298,-121.862630,Complaint of Pain,...,HNBD,Driver,35.0,Female,Slowing/Stopping,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,2.0
201800001515,1/12/2018,13:51,Friday,VALLEY AV,BERNAL AV (W),6157279.978,2065623.979,37.659538,-121.896997,Other Visible Injury,...,HNBD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
201900001307,1/12/2019,18:49,Saturday,VALLEY AV,BERNAL AV (W),6157276.000,2065234.000,37.658467,-121.897011,Complaint of Pain,...,HNBD,Driver,33.0,Female,Proceeding Straight,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,1.0
201400002173,1/15/2014,17:17,Wednesday,VALLEY AV,PASEO SANTA CRUZ (S),6157139.897,2068381.548,37.666962,-121.897666,Property Damage Only,...,HNBD,Driver,54.0,Male,Stopped In Road,Cell Phone Not In Use,None Apparent,NaN,Lap/Shoulder Harness Used,0.0
201600001644,1/15/2016,10:02,Friday,VALLEY AV,BOULDER ST,6168068.000,2070521.000,37.673381,-121.859995,Complaint of Pain,...,HNBD,Driver,58.0,Female,Making Left Turn,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,1.0
20180000246,1/17/2018,15:19,Wednesday,VALLEY AV,QUARRY LN,6166485.915,2071540.939,37.676182,-121.865461,Other Visible Injury,...,HNBD,Driver,47.0,Female,Stopped In Road,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,2.0
201200002748,1/20/2012,21:30,Friday,VALLEY AV,LAGUNA CREEK LN,6158915.382,2062869.911,37.651975,-121.891349,Property Damage Only,...,HNBD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
201600002234,1/20/2016,13:31,Wednesday,VALLEY AV,GREENWOOD RD,6162174.000,2071801.000,37.676695,-121.880376,Complaint of Pain,...,HNBD,Driver,67.0,Male,Proceeding Straight,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,1.0


#### Incidents on Valley Ave

In [63]:
road_injury_vallye_av_df = incidents_df[incidents_df["Primary Road"].isin(["VALLEY AV"])][["Primary Road","Secondary Road"]].groupby(["Secondary Road"]).count()
road_injury_vallye_av_df.columns
road_injury_vallye_av_df.columns = ["Primary_Valley_Av"]
road_injury_vallye_av_df.sort_values("Primary_Valley_Av",ascending=False)

,Primary_Valley_Av
Secondary Road,
SANTA RITA RD,45
BUSCH RD,22
QUARRY LN,19
GREENWOOD RD,14
STANLEY BL,14
HOPYARD RD,13
BERNAL AV (W),12
PASEO SANTA CRUZ (S),12
KOLLN ST,11


#### Incidents with Injuries

In [64]:
road_injury_df[road_injury_df["Injuries"]>=1].sort_values("Injuries",ascending=False)

,Date,Time,Collision Day of Week,Primary Road,Secondary Road,X,Y,Latitude,Longitude,Highest Degree of Injury,...,Party Sobriety 1 1,Party Type 2,Party Age 2,Party Sex 2,Movement Preceding Collision 2,special information 2,Associated Cause - Factor 1 2,Party Sobriety 1 2,Safety Equipment 2,Injuries
Report Number,,,,,,,,,,,,,,,,,,,,,
2014000322850,8/18/2014,13:16,Monday,VALLEY AV,BUSCH RD,6167240.000,2071281.000,37.675468,-121.862856,Other Visible Injury,...,HNBD,Driver,78.0,Male,Proceeding Straight,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,4.0
201000039001,8/17/2010,10:53,Tuesday,VALLEY AV,KOLLN ST,6165774.649,2071580.779,37.676143,-121.867963,Complaint of Pain,...,HNBD,Driver,76.0,Not Stated,Stopped in Road,Not Stated,Not Stated,HNBD,Lap/Shoulder Harness Used,3.0
201030997,7/2/2010,15:29,Friday,VALLEY AV,SANTA RITA RD,6163663.070,2071776.281,37.676680,-121.875259,Severe Injury,...,HNBD,Driver,47.0,Not Stated,Proceeding Straight,Not Stated,Not Stated,HNBD,Lap/Shoulder Harness Used,3.0
201800047731,12/19/2018,18:29,Wednesday,VALLEY AV,PASEO SANTA CRUZ (S),6157134.000,2068413.000,37.667049,-121.897686,Other Visible Injury,...,HNBD,Driver,38.0,Female,Proceeding Straight,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,3.0
20110030226,7/1/2011,08:53,Friday,VALLEY AV,PASEO SANTA CRUZ (S),6157134.000,2068413.000,37.667049,-121.897686,Complaint of Pain,...,HNBD,Driver,43.0,Female,Proceeding Straight,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,3.0
201700011313,3/27/2017,12:24,Monday,VALLEY AV,KOLL CENTER PW (N),6157210.111,2066909.000,37.663067,-121.897239,Complaint of Pain,...,HNBD,Driver,29.0,Male,Slowing/Stopping,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,3.0
201000010768,3/3/2010,09:13,Wednesday,VALLEY AV,GREENWOOD RD,6162704.919,2071791.856,37.676723,-121.878570,Complaint of Pain,...,HNBD,Driver,51.0,Not Stated,Stopped in Road,Not Stated,Not Stated,HNBD,Lap/Shoulder Harness Used,3.0
201400027063,7/10/2014,11:47,Thursday,VALLEY AV,PASEO SANTA CRUZ (N),6156897.000,2070981.000,37.674194,-121.898638,Complaint of Pain,...,HNBD,Driver,50.0,Female,Making Left Turn,Cell Phone Not In Use,None Apparent,HNBD,Lap/Shoulder Harness Used,3.0
201028388,6/16/2010,12:52,Wednesday,VALLEY AV,SANTA RITA RD,6163803.052,2071774.006,37.676674,-121.874775,Complaint of Pain,...,HNBD,Driver,26.0,Not Stated,Making Right Turn,Not Stated,Not Stated,HNBD,Lap/Shoulder Harness Used,2.0


#### Valley Ave Report

##### View Specific Columns

In [65]:
road_injury_date_time_df = road_injury_df[["Primary Road", "Secondary Road", "Date", "Time","Number Injured","Number Killed"]]
road_injury_date_time_df["Date"] = pd.to_datetime(road_injury_date_time_df["Date"][1:])
road_injury_date_time_df.sort_values(["Date","Time"],ascending=False)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Primary Road,Secondary Road,Date,Time,Number Injured,Number Killed
Report Number,,,,,,
201900003242,VALLEY AV,HOPYARD RD,2019-01-28,15:50,1,0.0
201900001307,VALLEY AV,BERNAL AV (W),2019-01-12,18:49,1,0.0
201800047731,VALLEY AV,PASEO SANTA CRUZ (S),2018-12-19,18:29,3,0.0
201800044771,VALLEY AV,HOPYARD RD,2018-11-25,20:37,0,0.0
201800043069,VALLEY AV,PASEO SANTA CRUZ (N),2018-11-11,08:59,1,0.0
201800042900,VALLEY AV,BLACKBIRD DR,2018-11-09,17:59,0,0.0
201800042031,VALLEY AV,PASEO SANTA CRUZ (S),2018-11-02,08:06,1,0.0
201800040768,VALLEY AV,CRESTLINE RD,2018-10-23,09:47,1,0.0
201800039629,VALLEY AV,GREENWOOD RD,2018-10-13,11:21,0,0.0


##### School Time Injuries on Valley Ave

In [66]:
schooltime_injury_valley_df = incidents_df.copy()
schooltime_injury_valley_df["SchoolHour"] = schooltime_injury_valley_df.apply(lambda x: is_schooltime(x['Date'],x['Time']),axis=1)

In [67]:
target_df = schooltime_injury_valley_df[["SchoolHour","Number Injured","Collision Type","Party at Fault","Party Type 1","Party Age 1","Party Type 2","Party Age 2","Primary Road","Secondary Road"]].sort_values(["Number Injured"],ascending=False)
target_df

,SchoolHour,Number Injured,Collision Type,Party at Fault,Party Type 1,Party Age 1,Party Type 2,Party Age 2,Primary Road,Secondary Road
Report Number,,,,,,,,,,
201000049196,0,7,Rear-End,NaN,Driver,54.0,Driver,53.0,STONERIDGE DR,JOHNSON DR
201400008137,1,6,Rear-End,1.0,Driver,17.0,Driver,18.0,LAS POSITAS BL,WILLOW RD
201300016522,1,5,Rear-End,1.0,Driver,52.0,Driver,38.0,STONERIDGE DR,OFF RAMP NB I680 TO STONERIDGE DR
201400048675,1,5,Broadside,NaN,Driver,44.0,Driver,37.0,STONERIDGE DR,CHABOT DR
201700029026,0,5,Broadside,1.0,Driver,18.0,Driver,62.0,FOOTHILL RD,STONERIDGE DR
201700012398,0,5,Broadside,1.0,Driver,64.0,Driver,60.0,HOPYARD RD,OWENS DR
201000046623,0,5,Rear-End,NaN,Driver,37.0,Driver,64.0,SANTA RITA RD,LOCKHART LN
201800002339,1,5,Rear-End,1.0,Driver,45.0,Driver,23.0,FIRST ST,STANLEY BL
201300019845,0,5,Head-On,1.0,Driver,18.0,Driver,66.0,HOPYARD RD,STONERIDGE DR


##### Pedestrian Involved Injuries on Valley Ave

In [68]:
target_df[(target_df["Party Type 1"] == "Pedestrian") | (target_df["Party Type 2"] == "Pedestrian")]

,SchoolHour,Number Injured,Collision Type,Party at Fault,Party Type 1,Party Age 1,Party Type 2,Party Age 2,Primary Road,Secondary Road
Report Number,,,,,,,,,,
201600026126,1,2,Vehicle - Pedestrian,1.0,Driver,NaN,Pedestrian,32.0,SANTA RITA RD,LAS POSITAS BL
201500044560,0,2,Vehicle - Pedestrian,1.0,Driver,38.0,Pedestrian,43.0,VALLEY AV,BLACKBIRD DR
201300036051,0,2,Vehicle - Pedestrian,1.0,Driver,49.0,Pedestrian,72.0,SANTA RITA RD,FRANCISCO ST (W)
201100040484,0,2,Vehicle - Pedestrian,1.0,Driver,81.0,Pedestrian,16.0,MAYWOOD DR,SPRINGDALE AV
201400011237,1,1,Vehicle - Pedestrian,1.0,Driver,NaN,Pedestrian,16.0,SANTA RITA RD,VALLEY AV
201800008354,0,1,Vehicle - Pedestrian,1.0,Driver,17.0,Pedestrian,5.0,MAIN ST,SPRING ST
201500010868,1,1,Vehicle - Pedestrian,1.0,Driver,84.0,Pedestrian,81.0,HANSEN DR,HOPYARD RD (N)
201600017739,1,1,Vehicle - Pedestrian,1.0,Driver,84.0,Pedestrian,28.0,WILLOW RD,OWENS DR
2013-00004900,1,1,Vehicle - Pedestrian,1.0,Driver,49.0,Pedestrian,54.0,GIBRALTAR DR,CHABOT DR


#####  Parties involved less than 16 years Age

In [69]:
target_df[(target_df["Party Age 1"] < 16) | (target_df["Party Age 2"] < 16)]

,SchoolHour,Number Injured,Collision Type,Party at Fault,Party Type 1,Party Age 1,Party Type 2,Party Age 2,Primary Road,Secondary Road
Report Number,,,,,,,,,,
20110019961,0,2,Broadside,2.0,Bicyclist,57.0,Bicyclist,12.0,AMARAL CI,KOTTINGER DR (W)
201800045589,0,2,Broadside,1.0,Driver,37.0,Bicyclist,13.0,LAS POSITAS BL,FIESTA DR
201000059310,1,2,Broadside,NaN,Driver,16.0,Bicyclist,15.0,VALLEY AV,GREENWOOD RD
201400012835,1,2,Broadside,1.0,Driver,29.0,Bicyclist,12.0,RAY ST,WALNUT DR
201400011834,1,1,Other,1.0,Bicyclist,9.0,NaN,NaN,TOURIGA DR,ARBOR DR
201400011776,1,1,Broadside,1.0,Driver,59.0,Bicyclist,12.0,WILLOW RD,GIBRALTAR DR (S)
201800008354,0,1,Vehicle - Pedestrian,1.0,Driver,17.0,Pedestrian,5.0,MAIN ST,SPRING ST
201400019940,1,1,Broadside,1.0,Bicyclist,15.0,Driver,48.0,LAS POSITAS BL,SANTA RITA RD
201013242,1,1,Overturned,NaN,Bicyclist,12.0,NaN,NaN,SANTA RITA RD,ALVARADO ST
